<a href="https://colab.research.google.com/github/LFelipes91/MVP-Machine_Learning_-_Analytics/blob/main/MVP_Machine_Learning_%26_Analytics_Luiz_Felipe_Silva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUC Rio - MVP Machine Learning & Analytics

Aluno: Luiz Felipe Silva Pereira

Sprint: 40530010056_20240_02

# Definição do Problema

**Objetivo: entender e descrever claramente o problema que está sendo resolvido.**

**Qual é a descrição do problema?**

▶ *Trata-se de um problema de Classificação. A proposta é criar um modelo de aprendizado supervisionado para classificar padrões de uso de dispositivos móveis (smartphones) e classificação do comportamento do usuário. Cada entrada é categorizada em uma das cinco classes de comportamento do usuário, variando de uso leve a extremo.*

 **Você tem premissas ou hipóteses sobre o problema? Quais?**

▶


 **Que restrições ou condições foram impostas para selecionar os dados?**

▶


 **Descreva o seu dataset (atributos, imagens, anotações, etc).**

▶ *Originalmente este dataset consiste em 10 atributos, mais o target "User Behavior Class".*

Link do Dataset: https://www.kaggle.com/datasets/valakhorasani/mobile-device-usage-and-user-behavior-dataset?resource=download


*Seguem atributos abaixo:*

▪ *ID do usuário: Identificador exclusivo para cada usuário.*

▪ *Modelo do dispositivo: Modelo do smartphone do usuário.*

▪ *Sistema operacional: O sistema operacional do dispositivo (iOS ou Android).*

▪ *Tempo de uso do aplicativo: tempo diário gasto em aplicativos móveis, medido em minutos.*

▪ *Tempo de tela ativa: média de horas por dia em que a tela fica ativa.*

▪ *Consumo de bateria: consumo diário de bateria em mAh.*

▪ *Número de aplicativos instalados: total de aplicativos disponíveis no dispositivo.*

▪ *Uso de dados: consumo diário de dados móveis em megabytes.*

▪ *Idade: Idade do usuário.*

▪ *Gênero: Gênero do usuário (masculino ou feminino).*

▪ *Classe de comportamento do usuário: Classificação do comportamento do usuário com base em padrões de uso (1 a 5).*



In [16]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine # para importar o dataset wine
from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste (holdout)
from sklearn.model_selection import KFold # para preparar os folds da validação cruzada
from sklearn.model_selection import cross_val_score # para executar a validação cruzada
from sklearn.metrics import accuracy_score # para a exibição da acurácia do modelo
from sklearn.neighbors import KNeighborsClassifier # algoritmo KNN
from sklearn.tree import DecisionTreeClassifier # algoritmo Árvore de Classificação
from sklearn.naive_bayes import GaussianNB # algoritmo Naive Bayes
from sklearn.svm import SVC # algoritmo SVM

In [10]:
# Carga do dataset

dataset = 'https://raw.githubusercontent.com/LFelipes91/MVP-Machine_Learning_-_Analytics/refs/heads/main/user_behavior_dataset.csv'

# importando dataset "user_behavior_dataset"
dados = pd.read_csv(dataset, delimiter = ',')

#wine = load_wine()
#dataset = pd.DataFrame(wine.data, columns=wine.feature_names) # conversão para dataframe
#dataset['target'] = wine.target # adição da coluna target

dados.head(10)

,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1,Google Pixel 5,Android,393,6.4,1872,67,1122,40,Male,4
1,2,OnePlus 9,Android,268,4.7,1331,42,944,47,Female,3
2,3,Xiaomi Mi 11,Android,154,4.0,761,32,322,42,Male,2
3,4,Google Pixel 5,Android,239,4.8,1676,56,871,20,Male,3
4,5,iPhone 12,iOS,187,4.3,1367,58,988,31,Female,3
5,6,Google Pixel 5,Android,99,2.0,940,35,564,31,Male,2
6,7,Samsung Galaxy S21,Android,350,7.3,1802,66,1054,21,Female,4
7,8,OnePlus 9,Android,543,11.4,2956,82,1702,31,Male,5
8,9,Samsung Galaxy S21,Android,340,7.7,2138,75,1053,42,Female,4
9,10,iPhone 12,iOS,424,6.6,1957,75,1301,42,Male,4


# Preparação de Dados

Objetivo: realizar operações de preparação dos dados.

Separe o dataset entre treino e teste (e validação, se aplicável).

Faz sentido utilizar um método de validação cruzada? Justifique se não utilizar.

In [12]:
# prompt: separate data in train and test

# Separação dos dados em treino e teste
X = dados_tse.drop('User Behavior Class', axis=1)
y = dados_tse['User Behavior Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 70% treino e 30% teste

Verifique quais operações de transformação de dados (como normalização e padronização, transformação de imagens em tensores) são mais apropriadas para o seu problema e salve visões diferentes do seu dataset para posterior avaliação dos modelos.
Refine a quantidade de atributos disponíveis, realizando o processo de feature selection de forma adequada.

# Modelagem e treinamento:

Objetivo: construir modelos para resolver o problema em questão.

Selecione os algoritmos mais indicados para o problema e dataset escolhidos, justificando as suas escolhas.
Há algum ajuste inicial para os hiperparâmetros?
O modelo foi devidamente treinado? Foi observado problema de underfitting?
É possível otimizar os hiperparâmetros de algum dos modelos? Se sim, faça-o, justificando todas as escolhas.
Há algum método avançado ou mais complexo que possa ser avaliado?
Posso criar um comitê de modelos diferentes para o problema (ensembles)?

In [13]:
# Preparação dos dados

# Separação em bases de treino e teste (holdout)
array = dataset.values
X = array[:,0:13] # atributos
y = array[:,13] # classe (target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7) # faz a divisão

# Criando os folds para a validação cruzada
num_particoes = 10 # número de folds da validação cruzada
kfold = KFold(n_splits=num_particoes, shuffle=True, random_state=7) # faz o particionamento em 10 folds

AttributeError: 'str' object has no attribute 'values'

In [ ]:
# Modelagem

# Definindo uma seed global para esta célula de código
np.random.seed(7)

# Listas para armazenar os modelos, os resultados e os nomes dos modelos
models = []
results = []
names = []

# Preparando os modelos e adicionando-os em uma lista
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Avaliando um modelo por vez
for name, model in models:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) # média e desvio padrão dos 10 resultados da validação cruzada
  print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure()
fig.suptitle('Comparação da Acurácia dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# Criando um modelo com todo o conjunto de treino
model = GaussianNB()
model.fit(X_train, y_train)

# Fazendo as predições com o conjunto de teste
predictions = model.predict(X_test)

# Estimando a acurácia no conjunto de teste
print(accuracy_score(y_test, predictions))

# Avaliação de Resultados:

Objetivo: analisar o desempenho dos modelos gerados em dados não vistos (com a base de teste)

Selecione as métricas de avaliação condizentes com o problema, justificando.
Treine o modelo escolhido com toda a base de treino, e teste-o com a base de teste.
Os resultados fazem sentido?
Foi observado algum problema de overfitting?
Compare os resultados de diferentes modelos.
Descreva a melhor solução encontrada, justificando.